In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 12 11:49:08 2019
@author: wangxinyi
"""
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 
from collections import defaultdict
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from dateutil import parser
from scipy import stats  
from scipy.stats import entropy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


conn= pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='root123',charset='utf8',db='car_evaluation')

# all_df = pd.read_sql_query(
'''
select 
id,avg(x) as c_x ,avg(y) as c_y , min(x) as min_x,max(x) as max_x , min(y) as min_y,max(y) as max_y ,
avg(v) as avg_v ,STD(v) as stdev_v, avg(d) as avg_d ,STD(d) as stdev_d, (max(str_to_date(concat('2019',t),'%Y%m%d %H:%i:%s'))- min(str_to_date(concat('2019',t),'%Y%m%d %H:%i:%s')) )/count(*) as t ,
 ty 
from tianchi_01_train group by id,ty order by id
'''
#,conn)


all_data_df=pd.read_sql_query('''
select id,x,y,v,d,t,ty,concat(x,'-',y)  as xy from tianchi_01_train  order by  id,t
''',conn)
#where id<100

In [2]:
def t_p(t):
    #return t.size
    return (parser.parse(max(t))-parser.parse(min(t))).total_seconds()/t.size

def entropy_wxy(x):
    index_x={}
    for c in x:
        if c in index_x:
            index_x[c]=index_x[c]+1
        else:
            index_x[c]=1
    vv=np.array(list(index_x.values()))    
    return entropy(vv, base=2)
def sub_num(x):
    return np.max(x)-np.min(x)

def distinct_count(xy):
    return len(set(xy))

def model_num(x):
    return np.mean(x.mode())

def month(x):
    #print(type(x))
    xx=[]
    for c in x:
        xx.append(c[0:2])
    return int(pd.Series(xx).mode()[0])

def day(x):
    #print(type(x))
    xx=[]
    for c in x:
        xx.append(c[5:7])
    return int(pd.Series(xx).mode()[0])

def distance_median(xy):
    test_f=[ [float(xx) for xx in  x.split("-")] for x in xy]
    ss=[]
    for i in range(len(test_f)-1):
        s=np.linalg.norm(np.array(test_f[i])-np.array(test_f[i+1]))  
        ss.append(s)
    return np.percentile(ss,0.5)

def per25(x):
    return np.percentile(x,0.25)

def per75(x):
    return np.percentile(x,0.75)

def kmeans_xy(xy):
    return xy.size



all_df=all_data_df.groupby(['id', 'ty']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })

In [4]:
all_df

x                                                          \
                 mean          amax          amin        median     model_num   
id   ty                                                                         
0    拖网  6.119351e+06  6.152038e+06  6.118352e+06  6.118352e+06  6.118352e+06   
1    拖网  6.091460e+06  6.102450e+06  6.049472e+06  6.102450e+06  6.102450e+06   
2    拖网  6.183011e+06  6.183191e+06  6.182482e+06  6.183090e+06  6.183090e+06   
3    拖网  5.239159e+06  5.287805e+06  5.228590e+06  5.229843e+06  5.229843e+06   
4    围网  7.062005e+06  7.070797e+06  7.049394e+06  7.061679e+06  7.060750e+06   
5    拖网  6.488945e+06  6.503900e+06  6.388833e+06  6.496570e+06  6.496570e+06   
6    拖网  6.058099e+06  6.079949e+06  6.032505e+06  6.058808e+06  6.069079e+06   
7    围网  6.759621e+06  6.986976e+06  6.587996e+06  6.811245e+06  6.589755e+06   
8    拖网  6.201416e+06  6.215849e+06  6.181299e+06  6.203378e+06  6.199943e+06   
9    拖网  6.102853e+06  6.102853e+06  6.102853e+06  6.102853e+06  6.102853e+06   
10   拖网  6.262484e+06  6.346913e+06  6.246119e+06  6.246120e+06  6.246120e+06   
11   拖网  6.267081e+06  6.297563e+06  6.261950e+06  6.262057e+06  6.262056e+06   
12   拖网  5.318402e+06  5.352320e+06  5.300063e+06  5.316856e+06  5.316856e+06   
13   刺网  6.411739e+06  6.411755e+06  6.410865e+06  6.411754e+06  6.411754e+06   
14   拖网  6.246437e+06  6.246627e+06  6.246116e+06  6.246524e+06  6.246524e+06   
15   围网  6.892099e+06  6.906785e+06  6.876176e+06  6.896894e+06  6.883195e+06   
16   拖网  6.372926e+06  6.387452e+06  6.358638e+06  6.375693e+06  6.359136e+06   
17   拖网  6.289990e+06  6.353001e+06  6.234199e+06  6.314584e+06  6.234303e+06   
18   拖网  6.246623e+06  6.246626e+06  6.246524e+06  6.246625e+06  6.246625e+06   
19   围网  6.362218e+06  6.524222e+06  6.246226e+06  6.308271e+06  6.246830e+06   
20   围网  6.883487e+06  6.924227e+06  6.848396e+06  6.892475e+06  6.897458e+06   
21   拖网  6.254199e+06  6.277601e+06  6.242502e+06  6.246727e+06  6.246727e+06   
22   拖网  6.527676e+06  6.614717e+06  6.509778e+06  6.510065e+06  6.510065e+06   
23   围网  6.337415e+06  6.383786e+06  6.333488e+06  6.333588e+06  6.333588e+06   
24   拖网  6.201448e+06  6.228675e+06  6.150677e+06  6.202157e+06  6.150680e+06   
25   围网  6.790510e+06  6.826855e+06  6.747916e+06  6.795805e+06  6.751972e+06   
26   围网  6.460630e+06  6.485414e+06  6.413542e+06  6.476971e+06  6.413641e+06   
27   拖网  6.183090e+06  6.183191e+06  6.182987e+06  6.183090e+06  6.183090e+06   
28   拖网  6.119111e+06  6.154703e+06  6.102140e+06  6.117539e+06  6.102653e+06   
29   拖网  6.255823e+06  6.296629e+06  6.231837e+06  6.246181e+06  6.236674e+06   
...               ...           ...           ...           ...           ...   
6970 拖网  6.244386e+06  6.328222e+06  6.233307e+06  6.233408e+06  6.233408e+06   
6971 围网  6.444586e+06  6.502016e+06  6.363326e+06  6.465371e+06  6.363525e+06   
6972 拖网  6.186926e+06  6.210852e+06  6.162059e+06  6.187065e+06  6.180907e+06   
6973 拖网  6.141704e+06  6.141805e+06  6.141701e+06  6.141703e+06  6.141703e+06   
6974 拖网  6.246435e+06  6.249220e+06  6.245823e+06  6.246425e+06  6.246425e+06   
6975 拖网  6.251329e+06  6.306865e+06  6.246229e+06  6.246828e+06  6.246828e+06   
6976 围网  6.965761e+06  7.038243e+06  6.947172e+06  6.958237e+06  6.952636e+06   
6977 拖网  6.118386e+06  6.118457e+06  6.118354e+06  6.118354e+06  6.118354e+06   
6978 围网  7.064309e+06  7.070480e+06  7.056510e+06  7.064971e+06  7.065652e+06   
6979 刺网  6.358977e+06  6.359063e+06  6.358962e+06  6.358962e+06  6.358962e+06   
6980 围网  6.455519e+06  6.473924e+06  6.416700e+06  6.467941e+06  6.416799e+06   
6981 拖网  6.285961e+06  6.327711e+06  6.234007e+06  6.309151e+06  6.234108e+06   
6982 拖网  6.068185e+06  6.087619e+06  6.051396e+06  6.069087e+06  6.075384e+06   
6983 围网  6.940499e+06  6.977323e+06  6.927512e+06  6.935613e+06  6.957469e+06   
6984 拖网  6.181551e+06  6.225125e+06  6.109007e+06  6.208309e+06  6.109007e+06   
6985 拖网  6.178148e+06  6.210977e+06  6.150178e+06  

In [3]:
names_x_x=['c_x','min_x','max_x','median_x','model_x','sub_x','per25_x','per75_x',
           'c_y','min_y','max_y','median_y','model_y','sub_y','per25_y','per75_y',
           'avg_v','stdev_v','median_v','entropy_v','model_v','d_v','per25_v','per75_v',
           'avg_d','stdev_d','median_d','entropy_d','model_d','d_d','per25_d','per75_d',
           't','t_month','t_day',
           'd_xy','dm_xy']

names_y='ty'
all_df.columns=names_x_x

all_df=all_df.reset_index()

In [5]:
all_df.head(1)

,id,ty,c_x,min_x,max_x,median_x,model_x,sub_x,per25_x,per75_x,...,entropy_d,model_d,d_d,per25_d,per75_d,t,t_month,t_day,d_xy,dm_xy
0,0,拖网,6.119351e+06,6.152038e+06,6.118352e+06,6.118352e+06,6.118352e+06,33686.667453,6.118352e+06,6.118352e+06,...,0.439489,0.0,15,0.0,0.0,228728.818841,11,15,23,0.0


In [6]:
d = defaultdict(LabelEncoder)
df = all_df.apply(lambda x:d[x.name].fit_transform(x) if type(x[0]) is str or math.isnan(x[0])  else x)

In [71]:
names_x=['c_x','min_x','max_x','median_x','model_x','sub_x','per25_x','per75_x',
           'c_y','min_y','max_y','median_y','model_y','sub_y','per25_y','per75_y',
           'avg_v','stdev_v','median_v','entropy_v','model_v','d_v','per25_v','per75_v',
           'avg_d','stdev_d','median_d','entropy_d','model_d','d_d','per25_d','per75_d',
           't','t_month','t_day',
           'd_xy','dm_xy']

#scaler = StandardScaler()
#x_train= scaler.fit_transform(df[names_x])

x_train = df.loc[:,names_x].values

y_train = df.loc[:,names_y].values

In [72]:
x_train

array([[6.11935124e+06, 6.15203843e+06, 6.11835176e+06, ...,
        1.50000000e+01, 2.30000000e+01, 0.00000000e+00],
       [6.09146039e+06, 6.10244988e+06, 6.04947187e+06, ...,
        1.80000000e+01, 1.38000000e+02, 0.00000000e+00],
       [6.18301061e+06, 6.18319102e+06, 6.18248219e+06, ...,
        1.70000000e+01, 4.00000000e+00, 0.00000000e+00],
       ...,
       [5.24022168e+06, 5.28746208e+06, 5.22870046e+06, ...,
        7.00000000e+00, 9.60000000e+01, 0.00000000e+00],
       [6.37342539e+06, 6.38762489e+06, 6.36501974e+06, ...,
        2.30000000e+01, 1.71000000e+02, 0.00000000e+00],
       [6.23542746e+06, 6.24031719e+06, 6.20637253e+06, ...,
        1.00000000e+01, 5.40000000e+01, 0.00000000e+00]])

In [73]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV

calibrated_forest = CalibratedClassifierCV( base_estimator=KNeighborsClassifier(n_neighbors=3))
param_grid = {  'base_estimator__n_neighbors': [6,7],
              'base_estimator__weights':['distance'],
              'base_estimator__algorithm':['ball_tree'],
              'base_estimator__p':[2],
              'base_estimator__leaf_size':[8,10]
             }
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=0)
search = GridSearchCV(calibrated_forest, param_grid, cv=cv,scoring='f1_macro')  #'precision_macro', 'recall_macro','f1_macro'
search.fit( x_train, y_train)



GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.1, train_size=None),
             error_score=nan,
             estimator=CalibratedClassifierCV(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                                                  leaf_size=30,
                                                                                  metric='minkowski',
                                                                                  metric_params=None,
                                                                                  n_jobs=None,
                                                                                  n_neighbors=3,
                                                                                  p=2,
                                                                                  weights='uniform'),
                                              cv=None, method='sigmoid'),
             iid='depreca

In [74]:
search.best_score_
# 0.7162070408882731 scaler = StandardScaler() x_train= scaler.fit_transform(df[names_x])
# 0.809814787197832 不标准化

0.809814787197832

In [70]:
best_parameters = search.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))

	base_estimator: KNeighborsClassifier(algorithm='ball_tree', leaf_size=8, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='distance')
	base_estimator__algorithm: 'ball_tree'
	base_estimator__leaf_size: 8
	base_estimator__metric: 'minkowski'
	base_estimator__metric_params: None
	base_estimator__n_jobs: None
	base_estimator__n_neighbors: 6
	base_estimator__p: 2
	base_estimator__weights: 'distance'
	cv: None
	method: 'sigmoid'


In [30]:
p_y= search.predict(x_train)
df['p_ty']=p_y
f1_score(p_y,y_train,average='macro')

1.0

In [150]:
y_train

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [11]:
err_df=df[df['ty']!=df['p_ty']]
err_df.to_csv('err_df.csv')

In [52]:
all_test_df = pd.read_sql_query('''select id,x,y,v,d,t,concat(x,'-',y)  as xy from tianchi_01_test  order by  id,t  ''',conn)


test_df=all_test_df.groupby(['id']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })
test_df.columns=names_x_x
test_df=test_df.reset_index()

In [30]:
test_df

,id,c_x,min_x,max_x,median_x,model_x,c_y,min_y,max_y,median_y,...,model_v,d_v,avg_d,stdev_d,median_d,entropy_d,model_d,d_d,t,d_xy
0,7000,7.092492e+06,7.119130e+06,7.069441e+06,7.087014e+06,7.083957e+06,5.918416e+06,5.966537e+06,5.893360e+06,5.918174e+06,...,0.110,63.0,137.356568,115.674525,126.0,6.080001,0.0,147,6.091312e+06,160
1,7001,6.239019e+06,6.254992e+06,6.216428e+06,6.240004e+06,6.246625e+06,5.212498e+06,5.242126e+06,5.176732e+06,5.213941e+06,...,0.110,76.0,149.606987,104.616537,185.0,6.180437,40.0,144,1.380885e+05,367
2,7002,6.678498e+06,6.745702e+06,6.566008e+06,6.670449e+06,6.696588e+06,5.482646e+06,5.567766e+06,5.385267e+06,5.457670e+06,...,0.700,89.0,159.436585,113.566486,153.0,6.373595,50.0,163,1.540428e+05,396
3,7003,6.160467e+06,6.205663e+06,6.150675e+06,6.150875e+06,6.150875e+06,5.202435e+06,5.206247e+06,5.174729e+06,5.206031e+06,...,0.110,49.0,122.242353,119.624570,100.0,5.419250,0.0,147,2.228096e+05,71
4,7004,6.357542e+06,6.407177e+06,6.348864e+06,6.349065e+06,6.349065e+06,5.398000e+06,5.406460e+06,5.343990e+06,5.406460e+06,...,0.110,59.0,123.839196,116.631778,116.0,5.805988,0.0,158,2.379218e+05,100
5,7005,6.322045e+06,6.322055e+06,6.321954e+06,6.322055e+06,6.322055e+06,5.381925e+06,5.381943e+06,5.381833e+06,5.381942e+06,...,0.000,13.0,80.019802,109.330694,0.0,4.097435,0.0,112,1.565428e+05,4
6,7006,7.054221e+06,7.067504e+06,7.051061e+06,7.054275e+06,7.052163e+06,5.746713e+06,5.783325e+06,5.734358e+06,5.743727e+06,...,2.725,49.0,148.895570,98.147307,172.5,6.783413,178.0,141,2.996635e+05,286
7,7007,6.073201e+06,6.102651e+06,6.022657e+06,6.073013e+06,6.102649e+06,5.053933e+06,5.112762e+06,5.002226e+06,5.037612e+06,...,0.000,63.0,105.063260,106.541190,72.0,5.738276,0.0,163,5.528127e+06,280
8,7008,6.325893e+06,6.336889e+06,6.325721e+06,6.325721e+06,6.325721e+06,5.286243e+06,5.286323e+06,5.284157e+06,5.286256e+06,...,0.160,22.0,103.948837,111.857582,76.0,5.541970,0.0,187,1.468784e+05,10
9,7009,6.572146e+06,6.574011e+06,6.564846e+06,6.573241e+06,6.573241e+06,5.479199e+06,5.501269e+06,5.455010e+06,5.481878e+06,...,0.650,41.0,150.479714,103.255786,155.0,7.573890,0.0,240,1.507346e+05,148


In [75]:
# x_test = scaler.fit_transform(test_df[names_x])
x_test = test_df.loc[:,names_x].values

test_y_p= search.predict_proba(x_test)

In [54]:
x_test = test_df.loc[:,names_x].values
test_y= search.predict(x_test)
test_df['p_ty']=test_y
test_df['p_ty_str']=d['ty'].inverse_transform(test_y)
test_df['proba_0']=test_y_p[:,0]
test_df['proba_1']=test_y_p[:,1]
test_df['proba_2']=test_y_p[:,2]

In [55]:
test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]

,id,p_ty_str,p_ty,proba_0,proba_1,proba_2
0,7000,围网,1,0.029375,0.922732,0.047894
1,7001,拖网,2,0.044678,0.037702,0.917620
2,7002,围网,1,0.029657,0.885140,0.085203
3,7003,拖网,2,0.040819,0.037593,0.921587
4,7004,围网,1,0.029524,0.900886,0.069590
5,7005,拖网,2,0.449702,0.038231,0.512066
6,7006,围网,1,0.115559,0.825555,0.058886
7,7007,拖网,2,0.029209,0.037491,0.933300
8,7008,围网,1,0.047377,0.904069,0.048554
9,7009,拖网,2,0.030322,0.257067,0.712612


In [57]:
result_df=test_df[['id','p_ty_str']]
result_df.to_csv("tianchi01_result_0202_knn.csv",index=False,header=0)

In [56]:
result_df_p=test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]
result_df_p['proba_max']=result_df_p[['proba_0','proba_1','proba_2']].max(axis=1)
result_df_p=result_df_p[['id','p_ty_str','proba_max']]
result_df_p.to_csv("tianchi01_result_p_0202_knn.csv",index=False,header=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
